<a href="https://colab.research.google.com/github/alicetech-dev/todo-app/blob/main/Lora_with_Flux_Comfyui_on_Google_Colab_Free.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Notes
- Uncheck `UPDATE_COMFY_UI`, and `USE_COMFYUI_MANAGER` and `INSTALL_GGUF_NODE` after first run if `USE_GOOGLE_DRIVE` is checked.


### Install Dependencies

In [1]:
# #@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = False  #@param {type:"boolean"}
INSTALL_GGUF_NODE = False  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_GGUF_NODE'] = INSTALL_GGUF_NODE
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull


if OPTIONS['INSTALL_GGUF_NODE']:
  %cd $WORKSPACE
  %cd custom_nodes
  !git clone https://github.com/city96/ComfyUI-GGUF
  !pip install -r ComfyUI-GGUF/requirements.txt

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies





Mounting Google Drive...
/
Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/ComfyUI
-= Install dependencies =-
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 MB 18.2 MB/s eta 0:00:00
   ━━━━━━

### Download Resources

In [ ]:
# Download the FLUX model from Hugging Face and save it in the ./models/unet directory
!wget -c "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q4_K_S.gguf" -P "./models/unet"

# Download the CLIP T5 model from Hugging Face and save it in ./models/clip
!wget -c "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q4_K_S.gguf" -P "./models/clip"

# Download the safetensors file for the CLIP model
!wget -c "https://huggingface.co/f5aiteam/CLIP/resolve/main/clip_l.safetensors" -P "./models/clip"

# Download the VAE model from Hugging Face and save it in ./models/vae
!wget -c "https://huggingface.co/f5aiteam/VAE/resolve/main/ae.safetensors" -P "./models/vae"

# Download the LoRa model from Civitai.com Replace `xxxxxxxx` with your actual token
!wget -c "https://civitai.com/api/download/models/1198851?type=Model&format=SafeTensor&token=xxxxxxxx" -O "./models/loras/FluxThouS40k.safetensors"


### Run ComfyUI

In [ ]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

Se truncaron las últimas líneas 5000 del resultado de transmisión.
lora key not loaded: lora_te2_text_model_encoder_layers_9_self_attn_out_proj.alpha
lora key not loaded: lora_te2_text_model_encoder_layers_9_self_attn_out_proj.lora_down.weight
lora key not loaded: lora_te2_text_model_encoder_layers_9_self_attn_out_proj.lora_up.weight
lora key not loaded: lora_te2_text_model_encoder_layers_9_self_attn_q_proj.alpha
lora key not loaded: lora_te2_text_model_encoder_layers_9_self_attn_q_proj.lora_down.weight
lora key not loaded: lora_te2_text_model_encoder_layers_9_self_attn_q_proj.lora_up.weight
lora key not loaded: lora_te2_text_model_encoder_layers_9_self_attn_v_proj.alpha
lora key not loaded: lora_te2_text_model_encoder_layers_9_self_attn_v_proj.lora_down.weight
lora key not loaded: lora_te2_text_model_encoder_layers_9_self_attn_v_proj.lora_up.weight
Requested to load SDXL
loaded completely 11830.873135375976 4897.0483474731445 True
100% 50/50 [00:41<00:00,  1.19it/s]
Prompt executed in

In [6]:
import os

PONY_MODEL_LOCAL_PATH = "/content/ponyDiffusionV6XL_v6StartWithThisOne.safetensors"
PONY_MODEL_URL = "https://huggingface.co/Magamanny/Pony-Diffusion-V6-XL/resolve/main/ponyDiffusionV6XL_v6StartWithThisOne.safetensors"

# Asegurar la descarga del modelo base Pony Diffusion
if not os.path.exists(PONY_MODEL_LOCAL_PATH):
    print(f"🔄 Descargando el modelo Pony Diffusion a {PONY_MODEL_LOCAL_PATH}...")
    # Usamos wget en lugar de aria2c
    # -O: guarda el archivo con el nombre especificado
    !wget -O "{PONY_MODEL_LOCAL_PATH}" "{PONY_MODEL_URL}"
    print("✅ Modelo Pony descargado.")
else:
    print(f"✅ Modelo Pony ya existe en {PONY_MODEL_LOCAL_PATH}.")

🔄 Descargando el modelo Pony Diffusion a /content/ponyDiffusionV6XL_v6StartWithThisOne.safetensors...
--2025-07-18 10:27:04--  https://huggingface.co/Magamanny/Pony-Diffusion-V6-XL/resolve/main/ponyDiffusionV6XL_v6StartWithThisOne.safetensors
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.37, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/a3/9c/a39c4dd29fc5f32795d65fd8ecdf4722875513c754feb55f50fdc87490d9e9c4/67ab2fd8ec439a89b3fedb15cc65f54336af163c7eb5e4f2acc98f090a29b0b3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27ponyDiffusionV6XL_v6StartWithThisOne.safetensors%3B+filename%3D%22ponyDiffusionV6XL_v6StartWithThisOne.safetensors%22%3B&Expires=1752838024&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MjgzODAyNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3